In [51]:
from pyspark.sql import *
from pyspark.sql.functions import *

In [2]:
path = '/data/'

In [3]:
geolocation = spark.read.csv(path+'olist_geolocation_dataset.csv',header=True,inferSchema = True)
order_items = spark.read.csv(path+'olist_order_items_dataset.csv',header=True,inferSchema = True)
payments = spark.read.csv(path+'olist_order_payments_dataset.csv',header=True,inferSchema = True)
reviews = spark.read.csv(path+'olist_order_reviews_dataset.csv',header=True,inferSchema = True)
orders = spark.read.csv(path+'olist_orders_dataset.csv',header=True,inferSchema = True)
products = spark.read.csv(path+'olist_products_dataset.csv',header=True,inferSchema = True)
seller = spark.read.csv(path+'olist_sellers_dataset.csv',header=True,inferSchema = True)
customer = spark.read.csv(path+'olist_customers_dataset.csv',header=True,inferSchema = True)

In [4]:
customer.show(10)

+--------------------+--------------------+------------------------+--------------------+--------------+
|         customer_id|  customer_unique_id|customer_zip_code_prefix|       customer_city|customer_state|
+--------------------+--------------------+------------------------+--------------------+--------------+
|06b8999e2fba1a1fb...|861eff4711a542e4b...|                   14409|              franca|            SP|
|18955e83d337fd6b2...|290c77bc529b7ac93...|                    9790|sao bernardo do c...|            SP|
|4e7b3e00288586ebd...|060e732b5b29e8181...|                    1151|           sao paulo|            SP|
|b2b6027bc5c5109e5...|259dac757896d24d7...|                    8775|     mogi das cruzes|            SP|
|4f2d8ab171c80ec83...|345ecd01c38d18a90...|                   13056|            campinas|            SP|
|879864dab9bc30475...|4c93744516667ad3b...|                   89254|      jaragua do sul|            SC|
|fd826e7cf63160e53...|addec96d2e059c80c...|            

In [5]:
# Caching some frequently used dataframe

orders.cache()
seller.cache()
order_items.cache()

DataFrame[order_id: string, order_item_id: int, product_id: string, seller_id: string, shipping_limit_date: timestamp, price: double, freight_value: double]

**Data Integration(all data frame)

In [6]:
all_dataframe = order_items.join(orders,'order_id','inner')

In [7]:
all_dataframe.show()

+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|order_item_id|          product_id|           seller_id|shipping_limit_date| price|freight_value|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|            1|87285b34884572647...|3504c0cb71d7fa48d...|2017-10-06 11:07:15| 29.99|         8.72|9ef432eb625129730

In [11]:
all_dataframe = all_dataframe.join(payments,'order_id','left')

In [12]:
all_dataframe.count()

117604

In [13]:
all_dataframe = all_dataframe.join(reviews,'order_id','left')

In [14]:
all_dataframe.count()

118310

In [16]:
all_dataframe = all_dataframe.join(customer,'customer_id','left')

In [18]:
all_dataframe.count()

118310

In [19]:
all_dataframe = all_dataframe.join(products,'product_id','left')

In [21]:
all_dataframe.count()

118310

In [62]:
all_dataframe = all_dataframe.join(broadcast(seller),'seller_id','left')

In [64]:
all_dataframe.count()

18064261

In [ ]:
all_dataframe = all_dataframe.join(geolocation,all_dataframe.customer_zip_code_prefix == geolocation.geolocation_zip_code_prefix,'left')

In [25]:
customer.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- customer_unique_id: string (nullable = true)
 |-- customer_zip_code_prefix: integer (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)



In [26]:
geolocation.printSchema()

root
 |-- geolocation_zip_code_prefix: integer (nullable = true)
 |-- geolocation_lat: double (nullable = true)
 |-- geolocation_lng: double (nullable = true)
 |-- geolocation_city: string (nullable = true)
 |-- geolocation_state: string (nullable = true)



In [35]:
all_dataframe.printSchema()

root
 |-- seller_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)
 |-- payment_sequential: integer (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- payment_installments: integer (nullable = true)
 |-- payment_value: double (nullable = true)
 |-- review_id: string (nullable = true)
 |-- review_score: string (nullable = true)
 |-- review_comment_tit

In [67]:
all_dataframe.cache()

25/12/25 08:06:42 WARN CacheManager: Asked to cache already cached data.


DataFrame[seller_id: string, product_id: string, customer_id: string, order_id: string, order_item_id: int, shipping_limit_date: timestamp, price: double, freight_value: double, order_status: string, order_purchase_timestamp: timestamp, order_approved_at: timestamp, order_delivered_carrier_date: timestamp, order_delivered_customer_date: timestamp, order_estimated_delivery_date: timestamp, payment_sequential: int, payment_type: string, payment_installments: int, payment_value: double, review_id: string, review_score: string, review_comment_title: string, review_comment_message: string, review_creation_date: string, review_answer_timestamp: string, customer_unique_id: string, customer_zip_code_prefix: int, customer_city: string, customer_state: string, product_category_name: string, product_name_lenght: int, product_description_lenght: int, product_photos_qty: int, product_weight_g: int, product_length_cm: int, product_height_cm: int, product_width_cm: int, seller_zip_code_prefix: int, s

#Total Order Value Per Customer
#Average Review Score per Seller 
#Most Sold Product (top 10)
#Top Customer Spending

In [82]:
#Total Order Per Customer
all_dataframe.groupBy('customer_id').agg(count('order_id')).orderBy('count(order_id)',ascending = False).show()

+--------------------+---------------+
|         customer_id|count(order_id)|
+--------------------+---------------+
|351e40989da90e704...|          11427|
|50920f8cd0681fd86...|          10752|
|9b43e2a62de9bab3a...|           8556|
|270c23a11d024a44c...|           8001|
|5c87184371002d49e...|           6876|
|d3e82ccec3cb5f956...|           6876|
|d5f2b3f597c7ccafb...|           6706|
|c2f18647725395af4...|           6612|
|24e7dc2ff8c071263...|           6597|
|7bb57d182bdc11653...|           6258|
|d22f25a9fadfb1abb...|           6072|
|63b964e79dee32a35...|           6072|
|1ff773612ab8934db...|           5820|
|13aa59158da63ba0e...|           5206|
|78fc46047c4a639e8...|           5200|
|dd3f1762eb601f41c...|           4992|
|a193aa8d905b8e246...|           4896|
|9eb3d566e87289dcb...|           4872|
|2ba91e12e5e4c9f56...|           4752|
|55e7cfd6e28d2fbfb...|           4728|
+--------------------+---------------+
only showing top 20 rows



In [61]:
#Average Review Score per Seller 
Seller_Average_Review  = all_dataframe.groupBy('seller_id').agg(avg('review_score')).orderBy('avg(review_score)', ascending = False).show()

+--------------------+-----------------+
|           seller_id|avg(review_score)|
+--------------------+-----------------+
|58c851d1a3c7cd3da...|              5.0|
|aa8af66c623d7d544...|              5.0|
|0f519b0d2e5eb2227...|              5.0|
|31e60bf8d103ce479...|              5.0|
|64c9a1db4e73e19aa...|              5.0|
|2b2fed75b8e5ea3a0...|              5.0|
|33ab10be054370c25...|              5.0|
|9d213f303afae4983...|              5.0|
|05a48cc8859962767...|              5.0|
|94d76e96eedd97625...|              5.0|
|e94b64dc6979b302a...|              5.0|
|4aba6a02a788d3ec8...|              5.0|
|fd312b6bf05efac6c...|              5.0|
|a61cc04793308395a...|              5.0|
|1a8e2d9c38b84a970...|              5.0|
|edb58a1390adf2738...|              5.0|
|c04d70d515d15a7e3...|              5.0|
|5f57db27027655e6c...|              5.0|
|c74f14c1e26cf1bd5...|              5.0|
|2c538755f1ca9540a...|              5.0|
+--------------------+-----------------+
only showing top

In [83]:
#Most sold product
all_dataframe.groupBy('product_id').count().orderBy('count',ascending=False).show()

+--------------------+-----+
|          product_id|count|
+--------------------+-----+
|aca2eb7d00ea1a7b8...|86740|
|422879e10f4668299...|81110|
|99a4788cb24856965...|78775|
|389d119b48cf3043d...|60248|
|d1c427060a0f73f6b...|59274|
|368c6c730842d7801...|58358|
|53759a2ecddad2bb8...|52654|
|53b36df67ebb7c415...|52105|
|154e7e31ebfa09220...|42700|
|3dd2a17168ec895c7...|40787|
|e53e557d5a159f5aa...|39516|
|2b4609f8948be1887...|36179|
|35afc973633aaeb6b...|31206|
|e0d64dcfaa3b6db5c...|31153|
|42a2c92a0979a949c...|30486|
|7c1bd920dbdf22470...|29018|
|a62e25e09e05e6faf...|28898|
|5a848e4ab52fd5445...|28737|
|c4baedd846ed09b85...|28166|
|b532349fe46b38fbc...|27176|
+--------------------+-----+
only showing top 20 rows



In [91]:
#top customerngin terms of spending
all_dataframe.groupBy('customer_id').agg(sum('payment_value')).orderBy('sum(payment_value)',ascending = False).show(10)

+--------------------+--------------------+
|         customer_id|  sum(payment_value)|
+--------------------+--------------------+
|1ff773612ab8934db...| 1.756825199999893E7|
|05455dfa7cd02f13d...|1.3282083359999327E7|
|ec5b2ba62e5743423...|1.0388528640000112E7|
|0c792d32a3251b4f6...|   8254681.600000529|
|78fc46047c4a639e8...|   7488519.999999339|
|1617b1357756262bf...|   7433259.520000033|
|1dbc055ccab23ed89...|   7216273.400000708|
|d5f2b3f597c7ccafb...|   6800018.119998923|
|dd3f1762eb601f41c...|  6746388.4800006235|
|10de381f8a8d23fff...|   5184499.500000076|
+--------------------+--------------------+
only showing top 10 rows



**Window Functions and Ranking

In [95]:
#Dense Rank for seller base on revenue
window_spec = Window.partitionBy('seller_id').orderBy(desc('price'))

In [96]:
#Rank Top Selling Products Per Seller
top_seller_products = all_dataframe.withColumn('rank', rank().over(window_spec)).filter(col('rank' ) < 5)

In [98]:
top_seller_products.select('seller_id', 'price', 'rank').show(5)

+--------------------+-----+----+
|           seller_id|price|rank|
+--------------------+-----+----+
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
+--------------------+-----+----+
only showing top 5 rows



**Advance Aggregation and Enrichment

In [ ]:
#total revenue and average order value (AOV) per customer

In [99]:
customer_spending = all_dataframe.groupBy('customer_id')\
.agg(
    count('order_id').alias('total_orders'),
    sum('price').alias('total_spent'),
    round(avg('price'),2).alias('AOV')

)\
.orderBy(desc('total_spent'))

In [100]:
customer_spending.show()

+--------------------+------------+------------------+-------+
|         customer_id|total_orders|       total_spent|    AOV|
+--------------------+------------+------------------+-------+
|d3e82ccec3cb5f956...|        6876|         6662844.0|  969.0|
|df55c14d1476a9a34...|         743|         3565657.0| 4799.0|
|fe5113a38e3575c04...|        2292|         3293604.0| 1437.0|
|ec5b2ba62e5743423...|        1428|         2556120.0| 1790.0|
|63b964e79dee32a35...|        6072|         2501664.0|  412.0|
|46bb3c0b1a65c8399...|         748|         2336752.0| 3124.0|
|05455dfa7cd02f13d...|        2184| 2160194.400000087|  989.1|
|3690e975641f01bd0...|         802|         2124498.0| 2649.0|
|349509b216bd5ec11...|         743|         1923627.0| 2589.0|
|695476b5848d64ba0...|         687|1820543.1299999943|2649.99|
|73236a0796f53d60d...|         832|         1755520.0| 2110.0|
|cc803a2c412833101...|         762|         1676400.0| 2200.0|
|1ff773612ab8934db...|        5820|1658641.7999999512| 

**Seller Performance Metrics (Revenue, Average Review, Order Count)

In [103]:
seller_perf = all_dataframe.groupBy('seller_id')\
    .agg(
        count('order_id').alias('total_orders'),
        round(sum('price'),2).alias('revenue'),
        round(avg('review_score'),2).alias('review_score')

).orderBy(desc('total_orders')).show()

+--------------------+------------+-------------+------------+
|           seller_id|total_orders|      revenue|review_score|
+--------------------+------------+-------------+------------+
|4a3ca9315b744ce9f...|      330661|3.375957084E7|        3.77|
|1f50f920176fa81da...|      297292|1.649745444E7|        4.04|
|6560211a19b47992c...|      286539| 1.73159329E7|        3.86|
|da8622b14eb17ae28...|      264433|2.985766973E7|        3.98|
|cc419e0650a3c5ba7...|      256032| 1.47514645E7|        4.07|
|7c67e1448b00f6e96...|      233306|3.228232179E7|        3.42|
|955fee9216a65b617...|      232364|2.096441067E7|        4.04|
|1025f0e2d44d7041d...|      229587|2.293751852E7|        3.89|
|4869f7a5dfa277a7d...|      184587|3.613871732E7|        4.09|
|3d871de0142ce09b7...|      175876| 1.41845253E7|        4.15|
|ea8482cd71df3c196...|      171766|   5631174.03|        3.92|
|7a67c85e85bb2ce85...|      167231|2.031279489E7|        4.26|
|8b321bb669392f516...|      165671|    2950366.6|      

In [115]:
#Product Popularity  Metrics

product_metrics = (
    all_dataframe
        .groupBy('product_id')
        .agg(
            count('order_id').alias('total_order'),
            sum('price').alias('revenue'),
            round(avg('price'),2).alias('avg_price'),
            round(stddev('price'),2).alias('price_volatilty'),
            collect_set('seller_id').alias('unique_sellers')
        )
        .orderBy('total_order', ascending=False)
)

product_metrics.show()

+--------------------+-----------+------------------+---------+---------------+--------------------+
|          product_id|total_order|           revenue|avg_price|price_volatilty|      unique_sellers|
+--------------------+-----------+------------------+---------+---------------+--------------------+
|aca2eb7d00ea1a7b8...|      86740|6164630.3000000175|    71.07|           3.17|[955fee9216a65b61...|
|422879e10f4668299...|      81110| 4442791.510000014|    54.77|           4.46|[1f50f920176fa81d...|
|99a4788cb24856965...|      78775| 6921762.710000017|    87.87|           4.08|[4a3ca9315b744ce9...|
|389d119b48cf3043d...|      60248|  3280533.13000001|    54.45|           4.37|[1f50f920176fa81d...|
|d1c427060a0f73f6b...|      59274|  8220103.32999999|   138.68|          16.58|[a1043bafd471dff5...|
|368c6c730842d7801...|      58358|3181698.8999999943|    54.52|           4.59|[1f50f920176fa81d...|
|53759a2ecddad2bb8...|      52654|2893017.4999999967|    54.94|           4.52|[1f50f920176

In [ ]:
#monthly revenue and order count trend

In [119]:
from pyspark.sql.functions import first, last


In [129]:
#customer retention analysis(showing the customer's first and last made orders)
customer_retention = all_dataframe.groupBy('customer_id').agg(first('order_purchase_timestamp').alias('first_order_stamp'),
                                                              last('order_purchase_timestamp').alias('last_order_stamp'),
                                                              count('order_id').alias('total_orders'),
                                                              round(avg('price'),2).alias('AOV')).orderBy(desc('total_orders'))
                                                              

In [ ]:
**Extended Enrichment

In [130]:
from pyspark.sql.functions import when

customer_spending = (
    customer_retention
        .withColumn(
            "customer_segment",
            when(col("AOV") >= 1200, "High_Value_CX")
            .when((col("AOV") < 1200) & (col("AOV") >= 500), "Medium_Value_CX")
            .otherwise("Low_Value_CX")
        )
)

customer_spending.show()


+--------------------+-------------------+-------------------+------------+------+----------------+
|         customer_id|  first_order_stamp|   last_order_stamp|total_orders|   AOV|customer_segment|
+--------------------+-------------------+-------------------+------------+------+----------------+
|351e40989da90e704...|2017-07-13 10:42:37|2017-07-13 10:42:37|       11427| 85.99|    Low_Value_CX|
|50920f8cd0681fd86...|2018-01-27 11:28:32|2018-01-27 11:28:32|       10752| 43.82|    Low_Value_CX|
|9b43e2a62de9bab3a...|2017-05-25 22:27:50|2017-05-25 22:27:50|        8556|  26.4|    Low_Value_CX|
|270c23a11d024a44c...|2017-08-08 20:26:31|2017-08-08 20:26:31|        8001| 36.59|    Low_Value_CX|
|5c87184371002d49e...|2018-01-05 19:15:37|2018-01-05 19:15:37|        6876| 12.49|    Low_Value_CX|
|d3e82ccec3cb5f956...|2017-03-18 14:28:34|2017-03-18 14:28:34|        6876| 969.0| Medium_Value_CX|
|d5f2b3f597c7ccafb...|2017-12-13 14:21:15|2017-12-13 14:21:15|        6706|  59.0|    Low_Value_CX|


In [131]:
all_dataframe = all_dataframe.join(customer_spending.select('customer_id','customer_segment'),'customer_id','left')

In [132]:
all_dataframe.groupBy('customer_segment').count().show()

+----------------+--------+
|customer_segment|   count|
+----------------+--------+
| Medium_Value_CX|  424368|
|    Low_Value_CX|17539225|
|   High_Value_CX|  100668|
+----------------+--------+



In [135]:
#Hourly Order Distribution
from pyspark.sql.functions import expr

all_dataframe = all_dataframe.withColumn(
    "hour_of_day",
    expr("hour(order_purchase_timestamp)")
)


In [138]:
all_dataframe.select('order_purchase_timestamp','hour_of_day').show()

+------------------------+-----------+
|order_purchase_timestamp|hour_of_day|
+------------------------+-----------+
|     2018-08-07 23:19:16|         23|
|     2018-08-07 23:19:16|         23|
|     2018-08-07 23:19:16|         23|
|     2018-08-07 23:19:16|         23|
|     2018-08-07 23:19:16|         23|
|     2018-08-07 23:19:16|         23|
|     2018-08-07 23:19:16|         23|
|     2018-08-07 23:19:16|         23|
|     2018-08-07 23:19:16|         23|
|     2018-08-07 23:19:16|         23|
|     2018-08-07 23:19:16|         23|
|     2018-08-07 23:19:16|         23|
|     2018-08-07 23:19:16|         23|
|     2018-08-07 23:19:16|         23|
|     2018-08-07 23:19:16|         23|
|     2018-08-07 23:19:16|         23|
|     2018-08-07 23:19:16|         23|
|     2018-08-07 23:19:16|         23|
|     2018-08-07 23:19:16|         23|
|     2018-08-07 23:19:16|         23|
+------------------------+-----------+
only showing top 20 rows



In [140]:
#Weekends VS Weekdays Order Distribution

from pyspark.sql.functions import when, dayofweek, col, lit

all_dataframe = all_dataframe.withColumn(
    "order_day_type",
    when(
        dayofweek(col("order_purchase_timestamp")).isin(1, 7),
        lit("weekend")
    ).otherwise(lit("weekdays"))
)


In [144]:
all_dataframe \
    .select("order_purchase_timestamp", "order_day_type") \
    .filter(col("order_day_type") == "weekend") \
    .show()


+------------------------+--------------+
|order_purchase_timestamp|order_day_type|
+------------------------+--------------+
|     2017-12-24 13:52:04|       weekend|
|     2017-12-24 13:52:04|       weekend|
|     2017-12-24 13:52:04|       weekend|
|     2017-12-24 13:52:04|       weekend|
|     2017-12-24 13:52:04|       weekend|
|     2017-12-24 13:52:04|       weekend|
|     2017-12-24 13:52:04|       weekend|
|     2017-12-24 13:52:04|       weekend|
|     2017-12-24 13:52:04|       weekend|
|     2017-12-24 13:52:04|       weekend|
|     2017-12-24 13:52:04|       weekend|
|     2017-12-24 13:52:04|       weekend|
|     2017-12-24 13:52:04|       weekend|
|     2017-12-24 13:52:04|       weekend|
|     2017-12-24 13:52:04|       weekend|
|     2017-12-24 13:52:04|       weekend|
|     2017-12-24 13:52:04|       weekend|
|     2017-12-24 13:52:04|       weekend|
|     2017-12-24 13:52:04|       weekend|
|     2017-12-24 13:52:04|       weekend|
+------------------------+--------

In [145]:
all_dataframe.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)
 |-- payment_sequential: integer (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- payment_installments: integer (nullable = true)
 |-- payment_value: double (nullable = true)
 |-- review_id: string (nullable = true)
 |-- review_score: string (nullable = true)
 |-- review_comment_tit